In [ ]:
import json
import re
from json import dumps
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.functions import lit, explode, from_json, col, current_date,current_timestamp, lit, row_number, when, decode, regexp_replace, trim
from pyspark.sql import Window
from pyspark.sql.types import StringType, StructType, StructField, ArrayType, DecimalType
import datetime 
import requests
import urllib3
from datetime import datetime,date
import pytz
import io
import glob
import os
import boto3
from pyspark.sql import SparkSession
import pyspark

: 

In [2]:
AWS_ACCESS_KEY_ID="admin"
AWS_SECRET_ACCESS_KEY="password"

In [3]:
import boto3
s3 = boto3.client('s3', endpoint_url='http://minio:9000',
                  aws_access_key_id=AWS_ACCESS_KEY_ID,
                  aws_secret_access_key=AWS_SECRET_ACCESS_KEY)
s3.list_buckets()

{'ResponseMetadata': {'RequestId': '186D866FE7999190',
  'HostId': 'dd9025bab4ad464b049177c95eb6ebf374d3b3fd1af9251148b658df7ac2e3e8',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'accept-ranges': 'bytes',
   'content-length': '540',
   'content-type': 'application/xml',
   'server': 'MinIO',
   'strict-transport-security': 'max-age=31536000; includeSubDomains',
   'vary': 'Origin, Accept-Encoding',
   'x-amz-bucket-region': 'us-east-1',
   'x-amz-id-2': 'dd9025bab4ad464b049177c95eb6ebf374d3b3fd1af9251148b658df7ac2e3e8',
   'x-amz-request-id': '186D866FE7999190',
   'x-content-type-options': 'nosniff',
   'x-ratelimit-limit': '1998',
   'x-ratelimit-remaining': '1998',
   'x-xss-protection': '1; mode=block',
   'date': 'Sat, 11 Oct 2025 19:27:24 GMT'},
  'RetryAttempts': 0},
 'Buckets': [{'Name': 'bronze',
   'CreationDate': datetime.datetime(2025, 10, 11, 18, 50, 3, 77000, tzinfo=tzlocal())},
  {'Name': 'gold',
   'CreationDate': datetime.datetime(2025, 10, 11, 19, 8, 23, 613000, tzinfo

In [4]:
CATALOG_URI = "http://nessie:19120/api/v1" 
WAREHOUSE = "s3://silver/"               
STORAGE_URI = "http://minio:9000"  

In [5]:
conf = (
    pyspark.SparkConf()
    .setAppName('silver_layer_transformation')
    
    # ═════════════════════════════════════════════════════════════
    # DEPENDENCIAS ICEBERG, NESSIE Y AWS
    # ═════════════════════════════════════════════════════════════
    .set('spark.jars.packages', 
         'org.apache.iceberg:iceberg-spark-runtime-3.5_2.12:1.5.0,'
         'org.projectnessie.nessie-integrations:nessie-spark-extensions-3.5_2.12:0.77.1,'
         'software.amazon.awssdk:bundle:2.24.8,'
         'software.amazon.awssdk:url-connection-client:2.24.8,'
         'org.apache.hadoop:hadoop-aws:3.2.0,'
         'com.amazonaws:aws-java-sdk-bundle:1.11.534')
    
    # ═════════════════════════════════════════════════════════════
    # CATÁLOGO ICEBERG CON NESSIE
    # ═════════════════════════════════════════════════════════════
    .set('spark.sql.extensions', 
         'org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions,'
         'org.projectnessie.spark.extensions.NessieSparkSessionExtensions')
    .set('spark.sql.catalog.nessie', 'org.apache.iceberg.spark.SparkCatalog')
    .set('spark.sql.catalog.nessie.uri', CATALOG_URI)
    .set('spark.sql.catalog.nessie.ref', 'main')
    .set('spark.sql.catalog.nessie.authentication.type', 'NONE')
    .set('spark.sql.catalog.nessie.catalog-impl', 'org.apache.iceberg.nessie.NessieCatalog')
    .set('spark.sql.catalog.nessie.warehouse', WAREHOUSE)
    .set('spark.sql.catalog.nessie.io-impl', 'org.apache.iceberg.aws.s3.S3FileIO')
    .set('spark.sql.catalog.nessie.s3.endpoint', STORAGE_URI)
    .set('spark.sql.catalog.nessie.s3.path-style-access', 'true')
    
    # ═════════════════════════════════════════════════════════════
    # CONFIGURACIÓN S3/MINIO (para lectura/escritura directa)
    # ═════════════════════════════════════════════════════════════
    .set('spark.hadoop.fs.s3a.endpoint', STORAGE_URI)
    .set('spark.hadoop.fs.s3a.access.key', AWS_ACCESS_KEY_ID)
    .set('spark.hadoop.fs.s3a.secret.key', AWS_SECRET_ACCESS_KEY)
    .set('spark.hadoop.fs.s3a.path.style.access', 'true')
    .set('spark.hadoop.fs.s3a.connection.ssl.enabled', 'false')
    .set('spark.hadoop.fs.s3a.impl', 'org.apache.hadoop.fs.s3a.S3AFileSystem')
    .set('spark.hadoop.fs.s3a.aws.credentials.provider',
                'org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider')
    
    # ═════════════════════════════════════════════════════════════
    # MEMORIA Y RENDIMIENTO
    # ═════════════════════════════════════════════════════════════
      .set('spark.driver.memory', '3g')      # Antes: 5g
    .set('spark.executor.memory', '4g')    # Antes: 8g
    .set('spark.sql.adaptive.enabled', 'true')  # Optimización
)

In [6]:
spark = SparkSession.builder \
    .config(conf=conf) \
    .getOrCreate()
print("Spark Session Started")

Spark Session Started


In [23]:
votes_2022 = spark.read.parquet("s3a://bronze/votes/2022/votes_2022.parquet")

In [24]:
votes_2022 = votes_2022.withColumn("load_date", current_timestamp())

In [25]:
votes_2023 = spark.read.parquet("s3a://bronze/votes/2023/votes_2023.parquet")

In [26]:
votes_2023 = votes_2023.withColumn("load_date", current_timestamp())

In [27]:
comments_2022 = spark.read.parquet("s3a://bronze/comments/_2022/1760209491.8317685.cc1709bc87.parquet")

In [28]:
comments_2022 = comments_2022.withColumn("load_date", current_timestamp())


In [29]:
comments_2023 = spark.read.parquet("s3a://bronze/comments/2023/comments_2023.parquet")

In [30]:
comments_2023 = comments_2023.withColumn("load_date", current_timestamp())

In [16]:
spark.sql("CREATE NAMESPACE IF NOT EXISTS nessie.silver").show()

++
||
++
++



In [17]:
votes_2022.printSchema()

root
 |-- Id: long (nullable = true)
 |-- PostId: long (nullable = true)
 |-- VoteTypeId: long (nullable = true)
 |-- CreationDate: timestamp (nullable = true)
 |-- UserId: long (nullable = true)
 |-- BountyAmount: decimal(20,0) (nullable = true)
 |-- load_date: timestamp (nullable = false)



In [18]:
comments_2022.printSchema()

root
 |-- id: long (nullable = true)
 |-- post_id: long (nullable = true)
 |-- score: long (nullable = true)
 |-- text: binary (nullable = true)
 |-- creation_date: timestamp (nullable = true)
 |-- user_id: long (nullable = true)
 |-- user_display_name: binary (nullable = true)
 |-- load_date: timestamp (nullable = false)



In [ ]:
def normalize_to_snake_case(df):
    """
    Convierte nombres de columna PascalCase a snake_case
    y elimina columnas técnicas que empiezan con '_dlt_'
    """


    def pascal_to_snake(name):
        """Convierte PascalCase a snake_case"""
        # Insertar _ antes de mayúsculas (excepto la primera)
        s1 = re.sub('(.)([A-Z][a-z]+)', r'\1_\2', name)
        # Insertar _ antes de mayúsculas seguidas de minúsculas
        return re.sub('([a-z0-9])([A-Z])', r'\1_\2', s1).lower()
    
    # Convertir columnas a snake_case
    renamed_count = 0
    for col_name in df.columns:
        # Saltar columnas técnicas
        if col_name.startswith('_dlt_'):
            continue
        
        # Si ya está en snake_case, saltar
        if col_name.islower() or '_' in col_name:
            continue
        
        # Convertir PascalCase a snake_case
        snake_case = pascal_to_snake(col_name)
        
        if snake_case != col_name:
            df = df.withColumnRenamed(col_name, snake_case)
            renamed_count += 1
    
    # Eliminar columnas técnicas
    dlt_columns = [c for c in df.columns if c.startswith('_dlt_')]
    if dlt_columns:
        df = df.drop(*dlt_columns)
        print(f"Removed technical columns: {', '.join(dlt_columns)}")
    
    if renamed_count > 0:
        print(f"Renamed {renamed_count} columns to snake_case")
    
    return df

In [ ]:
def clean_votes_dataframe(df, year):
    print(f"🧹 Limpiando Votes {year}...")

    # 🔸 Convertir tipo de dato
    df_clean = df.withColumn("BountyAmount", col("BountyAmount").cast(DecimalType(10, 2)))

    # 🔸 Definir ventana para deduplicar (con base en las columnas relevantes)
    window = (
        Window
        .partitionBy("PostId", "VoteTypeId", "UserId", "CreationDate")
        .orderBy("Id")
    )

    # 🔸 Identificar y eliminar duplicados
    df_dedup = (
        df_clean
        .withColumn("row_num", row_number().over(window))
        .filter(col("row_num") == 1)  # conserva solo el primero
        .drop("row_num")
    )

    # 🔸 Calcular estadísticas de limpieza
    original_count = df.count()
    dedup_count = df_dedup.count()
    duplicates_removed = original_count - dedup_count

    print(f"  ✓ Eliminados {duplicates_removed:,} votos duplicados de {original_count:,}")
    print(f"✅ Limpieza completa para Votes {year}")

    df = normalize_to_snake_case(df_dedup)

    return df


In [ ]:
def clean_comments_dataframe(df, year):

    print(f"Limpiando comments {year}...")

    df = normalize_to_snake_case(df)

    df_clean = (
        df
        .withColumn("text", decode(col("text"), "UTF-8"))
        .withColumn("user_display_name", decode(col("user_display_name"), "UTF-8"))
    )
    
    df_clean = (
        df_clean
        .withColumn("user_display_name", 
                    when(col("user_display_name").isNotNull(),
                         regexp_replace(trim(col("user_display_name")), r'\s+', ' '))
                    .otherwise(None))
    
        .withColumn("text",
                    when(col("text").isNotNull(),
                         regexp_replace(col("text"), r'[\x00-\x08\x0B-\x0C\x0E-\x1F\x7F]', ''))
                    .otherwise(None))
    )
    
    print(f"Limpieza para comments {year}")
    return df_clean

In [21]:
spark.sql("""
    CREATE TABLE IF NOT EXISTS nessie.silver.votes (
        Id BIGINT,
        PostId BIGINT,
        VoteTypeId BIGINT,
        CreationDate TIMESTAMP,
        UserId BIGINT,
        BountyAmount DECIMAL(10,2),
        load_date TIMESTAMP NOT NULL
    )
    USING iceberg
""")

DataFrame[]

In [ ]:
spark.sql("""
CREATE TABLE IF NOT EXISTS nessie.silver.comments (
    id BIGINT,
    post_id BIGINT,
    score BIGINT,
    text STRING,
    creation_date TIMESTAMP,
    user_id BIGINT,
    user_display_name STRING,
    load_date TIMESTAMP NOT NULL
)
USING iceberg
""")

: 

In [22]:
spark.stop()
spark = SparkSession.builder.config(conf=conf).getOrCreate()
print("✅ Spark reiniciado")

✅ Spark reiniciado


In [31]:
votes2022_clean=clean_votes_dataframe(votes_2022,2022)

🧹 Limpiando Votes 2022...
  ✓ Eliminados 1,447,233 votos duplicados de 15,851,534
✅ Limpieza completa para Votes 2022


In [32]:
votes2023_clean=clean_votes_dataframe(votes_2023,2023)

🧹 Limpiando Votes 2023...
  ✓ Eliminados 926,899 votos duplicados de 11,209,693
✅ Limpieza completa para Votes 2023


In [43]:
comments2022_clean=clean_comments_dataframe(comments_2022,2022)

Limpiando comments 2022...
Limpieza para comments 2022


In [44]:
comments2023_clean=clean_comments_dataframe(comments_2023,2023)

Limpiando comments 2023...


AnalysisException: [UNRESOLVED_COLUMN.WITH_SUGGESTION] A column or function parameter with name `user_display_name` cannot be resolved. Did you mean one of the following? [`UserDisplayName`, `UserId`, `load_date`, `CreationDate`, `Id`].;
'Project [Id#169L, PostId#170L, Score#171L, text#357, CreationDate#173, UserId#174L, UserDisplayName#175, load_date#183, decode('user_display_name, UTF-8) AS user_display_name#366]
+- Project [Id#169L, PostId#170L, Score#171L, decode(text#172, UTF-8) AS text#357, CreationDate#173, UserId#174L, UserDisplayName#175, load_date#183]
   +- Project [Id#169L, PostId#170L, Score#171L, Text#172, CreationDate#173, UserId#174L, UserDisplayName#175, current_timestamp() AS load_date#183]
      +- Relation [Id#169L,PostId#170L,Score#171L,Text#172,CreationDate#173,UserId#174L,UserDisplayName#175] parquet


In [ ]:
def merge_and_consolidate_votes(df_2022, df_2023):
    """Merge y consolidación de datos de Votes"""
    print("🔄 Realizando merge de Votes 2022 y 2023...")

    
    # Combinar DataFrames
    votes_combined = df_2022.unionByName(df_2023)
    
    # Estadísticas
    count_2022 = df_2022.count()
    count_2023 = df_2023.count()
    total_count = votes_combined.count()
    
    print(f"  📊 Estadísticas Votes:")
    print(f"    • 2022: {count_2022:,} registros")
    print(f"    • 2023: {count_2023:,} registros")
    print(f"    • Total: {total_count:,} registros")
    
    # Escribir en Iceberg
    votes_combined.writeTo("nessie.silver.votes").append()
    print("✅ Merge completado para Votes")
    
    return votes_combined

In [ ]:
def merge_and_consolidate_comments(df_2022, df_2023):
    """Merge y consolidación de datos de Comments"""
    print("🔄 Realizando merge de Comments 2022 y 2023...")
    
    # Agregar timestamp de carga si no existe
    if "load_date" not in df_2022.columns:
        df_2022 = df_2022.withColumn("load_date", current_timestamp())
    if "load_date" not in df_2023.columns:
        df_2023 = df_2023.withColumn("load_date", current_timestamp())
    
    # Combinar DataFrames
    comments_combined = df_2022.unionByName(df_2023)
    
    # Estadísticas
    count_2022 = df_2022.count()
    count_2023 = df_2023.count()
    total_count = comments_combined.count()
    
    print(f"  📊 Estadísticas Comments:")
    print(f"    • 2022: {count_2022:,} registros")
    print(f"    • 2023: {count_2023:,} registros")
    print(f"    • Total: {total_count:,} registros")
    
    # Escribir en Iceberg
    comments_combined.writeTo("nessie.silver.comments").append()
    print("✅ Merge completado para Comments")
    
    return comments_combined


: 

In [ ]:
print("INICIANDO PROCESO DE CONSOLIDACIÓN...")

votes_final = merge_and_consolidate_votes(votes2022_clean, votes2023_clean)
comments_final = merge_and_consolidate_comments(comments2022_clean, comments2023_clean)

print("\n CONSOLIDACIÓN FINALIZADA EXITOSAMENTE!")
print(f" Tablas creadas en Nessie Silver Layer:")
print(f"   • nessie.silver.votes: {votes_final.count():,} registros")
print(f"   • nessie.silver.comments: {comments_final.count():,} registros")